# Volcano plot visualisation

from https://github.com/MannLabs/ProteomicsVisualization

In [4]:
import pandas as pd
import re
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import datashader as ds
import plotly.io as pio
pio.renderers.default = "notebook"
import statsmodels.stats.power as power
import ext.easyMLR as emlr

In [5]:
def run_ttest(df: pd.DataFrame,
              c1: str, c2: str,
              cols_ann: list = ["Majority protein IDs", "Gene names", "Annotation"],
              s0: float or None = 0.05, fdr: float = 0.01, min_fc: float or None = None,
              n_perm: int = 2, plot_fdr_line: bool = True):
    """
    Run two-sided student's t-test with permutation based FDR control, either generating s0-based volcano lines or
    power-based quare cutoffs.
    
    Parameters
    ----------
    df: pandas DataFrame
        DataFrame containing data and annotations in pivot format
    c1, c2: regular expression string
        Regular expressions to extract columns containing replicate data for condition 1 and condition 2
    cols_ann: list, default = ["Majority protein IDs", "Gene names", "Annotation"]
        Additional annotation columns used for hover_annotation
    s0: float or None, default = 0.05
        If this is set and min_fc is None, non-linear volcano cutoff lines will be generated.
    fdr: float, default = 0.01
        False discovery rate for differentially expressed proteins.
    min_fc: float or None, default = None
        If the minimum fold change is set and s0 is None, square significance cutoffs will be used and the
        experimental power for the given fold change is calculated and reported.
    n_perm: int, default = 2
        Number of permutations to perform for FDR control. Recommended values are much higher.
    plot_fdr_line: bool, default = True
        Only effective for non-linear volcano lines. If true volcano lines are drawn. This is included because
        drawing these lines can takes some time.
    
    Returns
    -------
    a pandas DataFrame
        containing the selected data and annotation columns, and added columns for statistical output:
        fc, tval, pval, tval_s0, pval_s0, qval, FDR {fdr}%
    a plotly Scatter figure
        highlighting significant proteins and reporting FDR and s0 or power respectively.
    """
    
    # extract data
    df_in = df[[col for col in df.columns if col in cols_ann or re.match(c1, col) or re.match(c2, col)]].copy()
    df_in.set_index(cols_ann, inplace=True)
    df_in = df_in.apply(np.log2).replace([np.inf, -np.inf], np.nan).dropna().reset_index()
    
    # non-linear volcano lines
    if s0 and not min_fc:
        res, fig = emlr.perform_ttest_analysis(df_in, id_col=cols_ann[0],
                                               c1 = [col for col in df_in.columns if re.match(c1, col)],
                                               c2 = [col for col in df_in.columns if re.match(c2, col)],
                                               plot_fdr_line=plot_fdr_line, s0=s0, fdr=fdr, n_perm=n_perm)
        fig.update_layout(legend_title_text="FDR {}, s0 {}".format(str(fdr), str(s0)))
    
    # square cutoffs (first performs ttest with s0 = 0, then adds power analysis)
    elif not s0 and min_fc:
        res, _ = emlr.perform_ttest_analysis(df_in, id_col=cols_ann[0],
                                             c1 = [col for col in df_in.columns if re.match(c1, col)],
                                             c2 = [col for col in df_in.columns if re.match(c2, col)],
                                             plot_fdr_line=False, s0=0, fdr=fdr, n_perm=n_perm)
        res.insert(res.shape[1], "square_cutoff",
                   ["non_sig" if abs(fc)<min_fc or qval>fdr else "sig" for fc,qval in zip(res.fc, res.qval)])
        sd = max([df_in[[col for col in df_in.columns if re.match(c1, col)]].std(axis=1).mean(),
                  df_in[[col for col in df_in.columns if re.match(c2, col)]].std(axis=1).mean()])
        exp_pow = power.tt_ind_solve_power(effect_size=min_fc/sd, alpha=fdr,
                                           nobs1=len([col for col in df_in.columns if re.match(c1, col)]))
        fig = px.scatter(x=res.fc,
                         y=-np.log10(res.pval),
                         color=res.square_cutoff,
                         template='simple_white', render_mode="svg",
                         labels=dict(x="log2 fold change", y="-log10(p-value)",
                                     color="FDR {}, power {}".format(str(fdr), str(np.round(exp_pow, 2)))
                                    )
                        ).update_layout(width=600, height=700)\
        .add_vline(min_fc, line_width=2).add_vline(-min_fc, line_width=2)\
        .add_hline(-np.log10(res.loc[res.square_cutoff=="sig", "pval"].max()), line_width=2)
    return res, fig

## Load table

In [6]:
table = pd.read_csv(r'./proteins.csv')

In [8]:
list(table.columns)

['Protein Group',
 'Protein ID',
 'Accession',
 'Significance',
 '-10lgP',
 'Intensity iTRAQ8-113',
 'Intensity iTRAQ8-114',
 'Intensity iTRAQ8-115',
 'Intensity iTRAQ8-116',
 'Intensity iTRAQ8-117',
 'Intensity iTRAQ8-118',
 'Ratio iTRAQ8-113',
 'Ratio iTRAQ8-114',
 'Ratio iTRAQ8-115',
 'Ratio iTRAQ8-116',
 'Ratio iTRAQ8-117',
 'Ratio iTRAQ8-118',
 'Intensity WT(iTRAQ8-113; iTRAQ8-114; iTRAQ8-115)',
 'Intensity OE(iTRAQ8-116; iTRAQ8-117; iTRAQ8-118)',
 'Ratio WT(iTRAQ8-113; iTRAQ8-114; iTRAQ8-115)',
 'Ratio OE(iTRAQ8-116; iTRAQ8-117; iTRAQ8-118)',
 'Coverage (%)',
 '#Peptides',
 '#Unique',
 'PTM',
 'Avg. Mass',
 'Description']

In [9]:
table_drop = table.drop_duplicates('Protein Group')

In [16]:
table_drop_filt = table_drop[['Accession', 'Description',
       'Intensity iTRAQ8-113', 'Intensity iTRAQ8-114', 'Intensity iTRAQ8-115',
       'Intensity iTRAQ8-116', 'Intensity iTRAQ8-117', 'Intensity iTRAQ8-118']]

In [17]:
table_drop_filt.rename(columns={"Intensity iTRAQ8-113":"Intensity_wt1",
                               "Intensity iTRAQ8-114":"Intensity_wt2",
                               "Intensity iTRAQ8-115":"Intensity_wt3",
                               "Intensity iTRAQ8-116":"Intensity_oe1",
                               "Intensity iTRAQ8-117":"Intensity_oe2",
                               "Intensity iTRAQ8-118":"Intensity_oe3"}, inplace=True)

/Users/igorfesenko/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [18]:
table_drop_filt.head()

,Accession,Description,Intensity_wt1,Intensity_wt2,Intensity_wt3,Intensity_oe1,Intensity_oe2,Intensity_oe3
0,PhpapaCp032,NP_904195.1 atpB ATP synthase CF1 beta subunit,91230000.0,92410000.0,82530000.0,78890000.0,93300000.0,90830000.0
1,Pp3c18_21210V3.1.p,PREDICTED: 5-methyltetrahydropteroyltriglutama...,30250000.0,30200000.0,31400000.0,27010000.0,27980000.0,24190000.0
2,Pp3c15_12990V3.1.p,PREDICTED: lipoxygenase 3 chloroplastic [Bras...,76990000.0,75410000.0,80670000.0,91180000.0,93660000.0,87470000.0
4,Pp3c15_12980V3.1.p,unknown [Picea sitchensis].,11150000.0,12870000.0,13730000.0,12250000.0,12790000.0,12490000.0
5,Pp3c21_3730V3.1.p,na,9513000.0,8529000.0,9511000.0,9799000.0,9289000.0,7604000.0


## Statistics and visualisation

In [19]:
result_volc, fig_volc = run_ttest(table_drop_filt, cols_ann=["Accession", "Description"],
                                  c1="Intensity_wt*", c2="Intensity_oe*",
                                  s0=0.1, fdr=0.01, n_perm=10)

In [20]:
result_volc.head()

,Accession,Description,Intensity_wt1,Intensity_wt2,Intensity_wt3,Intensity_oe1,Intensity_oe2,Intensity_oe3,fc,tval,pval,tval_s0,pval_s0,qval,FDR 1%
0,PhpapaCp032,NP_904195.1 atpB ATP synthase CF1 beta subunit,26.443005,26.461546,26.298415,26.233339,26.475374,26.436666,0.019196,0.210785,0.843358,0.100466,0.924809,0.712926,non_sig
1,Pp3c18_21210V3.1.p,PREDICTED: 5-methyltetrahydropteroyltriglutama...,24.850432,24.848045,24.904261,24.686990,24.737893,24.527907,0.216649,3.289964,0.030217,1.306284,0.261506,0.189846,non_sig
2,Pp3c15_12990V3.1.p,PREDICTED: lipoxygenase 3 chloroplastic [Bras...,26.198168,26.168253,26.265529,26.442214,26.480930,26.382285,-0.224493,-5.525121,0.005241,-1.596324,0.185651,0.124923,non_sig
3,Pp3c15_12980V3.1.p,unknown [Picea sitchensis].,23.410540,23.617509,23.710828,23.546278,23.608513,23.574270,0.003272,0.036139,0.972903,0.017172,0.987122,0.782126,non_sig
4,Pp3c21_3730V3.1.p,na,23.181469,23.023945,23.181166,23.224203,23.147092,22.858327,0.052319,0.425042,0.692674,0.234519,0.826098,0.603689,non_sig


In [30]:
result_volc[result_volc['FDR 1%']=='sig']

,Accession,Description,Intensity_wt1,Intensity_wt2,Intensity_wt3,Intensity_oe1,Intensity_oe2,Intensity_oe3,fc,tval,pval,tval_s0,pval_s0,qval,FDR 1%
117,Pp3c1_1940V3.1.p,PREDICTED: jacalin-related lectin 3 isoform X2...,23.789550,23.858517,23.816166,23.355147,23.283480,23.205830,0.539925,11.351905,0.000343,3.658960,0.021600,0.0,sig
456,Pp3c3_14480V3.1.p,Rubisco SSU [Plantago major].,21.644384,21.587836,21.600595,22.291371,22.483454,22.351647,-0.764552,-12.904854,0.000208,-4.801095,0.008641,0.0,sig
925,Pp3c6_16850V3.1.p,uncharacterized protein CMC5_002860 [Chondromy...,22.331013,22.221698,22.360247,21.461140,21.123131,21.548397,0.926764,6.796228,0.002448,3.920910,0.017234,0.0,sig
1657,Pp3c15_18180V3.1.p,hypothetical protein OsJ_30403 [Oryza sativa J...,21.071038,21.002645,20.948065,20.398326,20.163001,20.294459,0.721987,9.398296,0.000714,4.083151,0.015059,0.0,sig


In [22]:
fig_volc.update_layout(font_size=10, width=600, height=800).show()

In [27]:
result_square, fig_square = run_ttest(table_drop_filt, cols_ann=["Accession", "Description"],
                                      c1="Intensity_wt*", c2="Intensity_oe*",
                                      min_fc=1.2, s0=None, fdr=0.01, n_perm=10)

In [28]:
result_square.head()

,Accession,Description,Intensity_wt1,Intensity_wt2,Intensity_wt3,Intensity_oe1,Intensity_oe2,Intensity_oe3,fc,tval,pval,tval_s0,pval_s0,qval,FDR 1%,square_cutoff
0,PhpapaCp032,NP_904195.1 atpB ATP synthase CF1 beta subunit,26.443005,26.461546,26.298415,26.233339,26.475374,26.436666,0.019196,0.210785,0.843358,0.210785,0.843358,0.282844,non_sig,non_sig
1,Pp3c18_21210V3.1.p,PREDICTED: 5-methyltetrahydropteroyltriglutama...,24.850432,24.848045,24.904261,24.686990,24.737893,24.527907,0.216649,3.289964,0.030217,3.289964,0.030217,0.000362,sig,non_sig
2,Pp3c15_12990V3.1.p,PREDICTED: lipoxygenase 3 chloroplastic [Bras...,26.198168,26.168253,26.265529,26.442214,26.480930,26.382285,-0.224493,-5.525121,0.005241,-5.525121,0.005241,0.000000,sig,non_sig
3,Pp3c15_12980V3.1.p,unknown [Picea sitchensis].,23.410540,23.617509,23.710828,23.546278,23.608513,23.574270,0.003272,0.036139,0.972903,0.036139,0.972903,0.367438,non_sig,non_sig
4,Pp3c21_3730V3.1.p,na,23.181469,23.023945,23.181166,23.224203,23.147092,22.858327,0.052319,0.425042,0.692674,0.425042,0.692674,0.184000,non_sig,non_sig


In [29]:
fig_square.update_layout(font_size=10, width=600, height=800).show()